This takes way too long. Bigger buffer?

In [1]:
# import json
# import sqlite3
# import numpy as np

# conn = sqlite3.Connection('data/data.db')
# cur = conn.cursor()

# create_embeddings = """
#     CREATE TABLE embeddings (
#         id INTEGER PRIMARY KEY,         
#         vector BLOB              
#     );
# """
# cur.execute(create_embeddings)
# conn.commit()

# insert_embeddings = """
#     INSERT INTO embeddings (id, vector) VALUES (?, ?);
# """

# buffer = []
# with open('data/embeddings.jsonl', 'rt') as file:
#     for line in file:
#         j = json.loads(line)
#         buffer.append((
#             j['id'], 
#             np.array(j['embedding']).tobytes()
#         ))
#         if len(buffer) > 100000:
#             print("Writing buffer.")
#             cur.executemany(
#                 insert_embeddings, 
#                 buffer
#             )
#             conn.commit()
#             buffer = []

# if len(buffer) > 0:
#     print("Writing buffer.")        
#     cur.executemany(
#         insert_embeddings, 
#         buffer
#     )
#     conn.commit()
#     buffer = []

# Connect counts and clean text.

In [2]:
import pandas as pd

selected = {}
with open('../transformed/data/links.txt', 'r', encoding='utf-8', errors='replace') as file:
    for line in file:
        try:
            # Filter out hashtag pages
            if line[0] == '#':
                continue

            # Filter out category pages
            skip = False
            for substring in ['Category:', 'File:', 'Wikipedia:']:
                if substring in line:
                    skip = True
            if skip:
                continue
                
            # Filter out aliases and sublinks
            for substring in ['#', '|']:
                if substring in line:
                    line = line.split(substring, 1)[0]

            line = line.strip()

            if line in selected:
                selected[line] += 1
            else:
                selected[line] = 1
        except Exception as e:
            print(e)

selected = pd.DataFrame(
    list(
        filter(
            lambda x: x['count'] >= 5,
            sorted(
                [{'lower_title': x.lower().strip(), 'count': y} for x, y in selected.items()],
                key=lambda x: x['count'],
                reverse=True
            )   
        )
    )
)

# Drop duplicates
selected = selected.sort_values('count', ascending=False)
selected = selected.drop_duplicates('lower_title', keep='first')

In [3]:
import json 
import pandas as pd

clean = []
with open('data/cleaned.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        clean.append(json.loads(line))
clean = pd.DataFrame(clean)

# Drop duplicates
clean['len'] = clean.clean_text.apply(len)
clean = clean.sort_values('len', ascending=False)
clean = clean.drop_duplicates('lower_title', keep='first')
clean = clean.drop('len', axis=1)

In [4]:
print(len(clean))
clean = clean.merge(selected, on='lower_title')
print(len(clean))

550751
550751


In [5]:
clean.sample()

,id,title,lower_title,clean_text,count
499179,2177907,Garde Mobile,garde mobile,"Garde Mobile: The '''''Garde mobile''''' (""Mob...",7


In [6]:
import sqlite3
import pandas as pd

try:
    conn = sqlite3.Connection('data/data.db')
    (clean
        .rename(columns={'lower_title': 'clean_title'})
        .drop(['clean_text'], axis=1)
        .to_sql('articles', conn)
    )
    conn.commit()
except Exception as e:
    print(e)
finally:
    conn.close()